# 요인분석 (FA)

## 요인분석이란?
* 요인분석(Factor Analysis, FA)은 관찰된 변수들의 숨겨진 요인을 찾아내는 통계 기법.
* 이러한 요인들은 관찰된 변수들 간의 공통적인 변동성을 설명하며, 데이터의 구조를 단순화하고 이해하는 데 도움을 준다.

## 요인분석이 필요한 이유
1. **데이터 차원 축소**: 많은 변수들 간의 상관관계를 소수의 요인으로 줄임으로써 데이터의 차원을 축소할 수 있다.
2. **해석 용이성**: 복잡한 데이터 구조를 간단하게 만들어 해석을 용이하게 한다.
3. **노이즈 감소**: 불필요한 변동성을 줄이고 중요한 패턴을 강조한다.
4. **변수 선택**: 중요한 변수를 식별하고 덜 중요한 변수를 제거하는 데 도움을 준다.

## 요인분석 진행 방법
1. **데이터 준비**: 분석할 데이터를 수집하고, 결측값을 처리하고, 변수를 정규화한다.
2. **상관 행렬 계산**: 변수들 간의 상관 관계를 계산하여 상관 행렬을 만든다.
3. **고유값 분해**: 상관 행렬의 고유값과 고유벡터를 계산하여 요인을 추출한다.
4. **요인 회전**: 요인의 해석 가능성을 높이기 위해 요인 회전을 수행한다 (예: 직교 회전, 사각 회전).
5. **요인 점수 계산**: 각 관측치에 대한 요인 점수를 계산한다.
6. **결과 해석**: 요인 패턴과 요인 점수를 해석하여 데이터의 구조를 이해한다.

## 요인분석과 주성분분석의 차이점
요인분석과 주성분분석(PCA)은 둘 다 차원 축소 기법이지만, 접근 방식과 목적이 다르다.

1. **목적**:
   - **요인분석**: 변수들 간의 상관관계를 설명하는 잠재 요인을 식별하는 데 중점을 둔다.
   - **주성분분석**: 데이터의 분산을 최대한으로 설명하는 주성분을 식별하는 데 중점을 둔다.

2. **모델**:
   - **요인분석**: 관찰된 변수들은 잠재 요인의 선형 결합으로 설명된다.
   - **주성분분석**: 주성분은 관찰된 변수들의 선형 결합으로 형성된다.

3. **결과 해석**:
   - **요인분석**: 요인 적재량(변수와 요인 간의 상관관계)을 통해 요인을 해석한다.
   - **주성분분석**: 주성분 적재량(변수와 주성분 간의 상관관계)을 통해 주성분을 해석한다.

# 패키지 없이 구현하기
* 패키지 없이 요인분석을 구현해보자.
* 단, 시각화 부분은 패키지를 사용

In [123]:
# 패키지 불러오기
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm
from sklearn.preprocessing import StandardScaler
from scipy.stats import multivariate_normal

In [124]:
# 시드값 설정
np.random.seed(2023)

# 자료 만들기
D = np.diag([1, 2, 0.4, 2])
data = [1, 0.6, 0.1, -0.1,
        0.6, 1, 0.2, -0.2,
        0.1, 0.2, 1, 0.7,
       -0.1, -0.2, 0.7, 1]
rr = np.array(data).reshape(4, 4)
Sigma = sqrtm(D) @ rr @ sqrtm(D)

n = 10000
mu = np.zeros(rr.shape[0])
np.random.seed(2023)
d1 = StandardScaler().fit_transform(multivariate_normal.rvs(mean=mu, cov=Sigma, size=n))

## 요인분석의 가정
* 관찰된 변수들 $X$가 잠재 요인 $F$의 선형 결합으로 표현된다고 가정.
$$X = \Delta F + \epsilon$$

여기서:
- $X$는 $p$개의 관찰된 변수들로 이루어진 벡터.
- $\Lambda$는 $p \times m$ 크기의 요인 적재 행렬 (Factor Loadings).
- $F$는 $m$개의 잠재 요인들로 이루어진 벡터.
- $\epsilon$은 오차 벡터로, 각 관찰된 변수의 고유 변동성.

즉, 변수들 $X$는 원인들 $F$의 선형결합이고, 그 가중치를 $\Lambda$로 표현.
* 변수들 $X$는 주어진 데이터이고, 원인들 $F$ 역시 정하기 나름이다.
* 우리가 구하려는 것은 $\Delta$와 $\epsilon$.

## 공분산행렬과 그 분해
* 변수들 $X$가 주어진 데이터일때, 그 공분산행렬 $R$은 다음과 같이 분해할 수 있다.

$$
R = \Delta \Delta^T + \Psi
$$

* 즉 $\Delta$와 $\epsilon$을 구하기 위해 $X$에서 $R$을 구한 뒤, 이를 분해한다.
* 성분 중 일부분을 이용해 $\Delta \Delta^T$로, 나머지 성분으로 $\Psi$를 만든다.

In [125]:
# 자료의 공분산행렬 R 도출
R = np.cov(d1, rowvar=False)
print("공분산 행렬 R:\n")
print(pd.DataFrame(R))

공분산 행렬 R:

          0         1         2         3
0  1.000100  0.591647  0.107398 -0.100142
1  0.591647  1.000100  0.215008 -0.193681
2  0.107398  0.215008  1.000100  0.697223
3 -0.100142 -0.193681  0.697223  1.000100


In [161]:
# 행렬 R의 고유값 분해
eigenvalues, eigenvectors = np.linalg.eigh(R)
idx = np.argsort(eigenvalues)[::-1]  # 고유값 내림차순 정렬 인덱스
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]
U = eigenvectors
L = np.diag(eigenvalues)

* 공분산 행렬의 고유값은 각 요인이 데이터의 총 분산에서 차지하는 비율을 나타낸다. 
* 즉, 고유값이 클수록 해당 요인은 중요한 정보를 담고 있다.

* 따라서, 행렬 R의 1, 2번째 성분과 3, 4번째 성분을 분리.
* 행렬 R의 1, 2번째 성분을 재구성해 만든 `Rm`
* 행렬 R의 3, 4번째 성분을 재구성해 만든 `Rmr`

In [162]:
# 행렬 R의 1, 2번째 성분을 사용하여 만든 행렬 Rm
m = 2
Um = U[:, :m]
Lm = L[:m, :m]
Rm = Um @ Lm @ Um.T
print("R의 1st, 2nd 성분 (Rm):\n")
print(pd.DataFrame(np.round(Rm, 3)))

R의 1st, 2nd 성분 (Rm):

       0      1      2      3
0  0.761  0.789  0.179 -0.174
1  0.789  0.819  0.195 -0.172
2  0.179  0.195  0.890  0.808
3 -0.174 -0.172  0.808  0.890


In [163]:
# R과 Rm의 차이
print("R - Rm = \n")
print(pd.DataFrame(np.round(R - Rm, 3)))

R - Rm = 

       0      1      2      3
0  0.239 -0.198 -0.072  0.074
1 -0.198  0.181  0.020 -0.022
2 -0.072  0.020  0.110 -0.110
3  0.074 -0.022 -0.110  0.110


In [164]:
# R의 3, 4번째 성분을 사용하여 만든 행렬 Rmr
Umr = U[:, m:]
Lmr = L[m:, m:]
Rmr = Umr @ Lmr @ Umr.T
print("R의 3rd, 4th 성분 (Rmr):\n")
print(pd.DataFrame(np.round(Rmr, 3)))

R의 3rd, 4th 성분 (Rmr):

       0      1      2      3
0  0.239 -0.198 -0.072  0.074
1 -0.198  0.181  0.020 -0.022
2 -0.072  0.020  0.110 -0.110
3  0.074 -0.022 -0.110  0.110


`R - Rm`과 `Rmr`이 같다는 것을 알 수 있다.

## $\Delta$와 $\Psi$ 구하기

$\Psi$는 대각행렬로 생각. 
- 각 변수의 고유 변동성이 서로 독립적이며, 다른 변수들과의 공분산이 0으로 가정한다는 뜻.
- 마치 다중회귀분석에서 오차항의 공분산이 $\sigma^2 I$인 것과 비슷한 논리
- 따라서 `Rmr`의 대각성분만을 추출해 대각행렬을 만들어 `psi`를 만듬

In [169]:
# psi 구하기
psi = np.diag(np.diag(Rmr))
psi = np.round(psi, 4)

print("\n대각 행렬 (psi):")
print(pd.DataFrame(psi))


대각 행렬 (psi):
       0       1       2       3
0  0.239  0.0000  0.0000  0.0000
1  0.000  0.1813  0.0000  0.0000
2  0.000  0.0000  0.1103  0.0000
3  0.000  0.0000  0.0000  0.1104


이후 $\Delta \Delta^T$ = `ddt`를 만듬.
* `ddt`를 다시 고유값 분해. 
* 주성분을 골라서 $\Delta \Delta^T$의 근사값을 도출

In [179]:
# ddt 만들어보기
ddT = R - psi

# 고유값 분해 후 다시 변수 축소
eigenvalues_2, eigenvectors_2 = np.linalg.eigh(ddT)
idx_2 = np.argsort(eigenvalues_2)[::-1]
eigenvalues_2 = eigenvalues_2[idx_2]
eigenvectors_2 = eigenvectors_2[:, idx_2]

# 설명비율로 변수 고르기
l1 = eigenvalues_2
print(l1)
explained_variance_ratio = np.cumsum(l1) / np.sum(l1)
print("설명비율:", explained_variance_ratio)

[1.58855549 1.45728909 0.25576095 0.05779451]
설명비율: [0.47286881 0.90666326 0.98279618 1.        ]


In [180]:
# 2개를 고르는 것으로 보고 근사값 도출
U_2 = eigenvectors_2[:, :2]
L_2 = np.diag(eigenvalues_2[:2])
approx_dd_T = U_2 @ L_2 @ U_2.T

In [181]:
# 원래의 R과 비교해보기
print("\n근사값 (approx.dd.T) + 대각 행렬 (Rmr):")
print(pd.DataFrame(np.round(approx_dd_T + np.diag(np.diag(Rmr)), 3)))

print("\n원래의 공분산 행렬 R:")
print(pd.DataFrame(np.round(R, 3)))


근사값 (approx.dd.T) + 대각 행렬 (Rmr):
       0      1      2      3
0  0.868  0.685  0.165 -0.159
1  0.685  0.927  0.188 -0.166
2  0.165  0.188  0.946  0.752
3 -0.159 -0.166  0.752  0.945

원래의 공분산 행렬 R:
       0      1      2      3
0  1.000  0.592  0.107 -0.100
1  0.592  1.000  0.215 -0.194
2  0.107  0.215  1.000  0.697
3 -0.100 -0.194  0.697  1.000


어느 정도 비슷한 값이 나온 것이 알 수 있다.  
이를 근거로 인제 $\Delta$를 구해보자

In [185]:
# delta 얻기
Delta = U @ np.sqrt(L)
pd.DataFrame(Delta)

,0,1,2,3
0,0.286872,-0.823884,0.484493,-0.065463
1,0.306612,-0.851368,-0.383023,0.185888
2,0.937033,0.108462,-0.185899,-0.275220
3,0.804943,0.491658,0.189635,0.272907


## 인자점수 도출하기
* 인자점수는 변수 $X$에 원인 $F$가 얼마나 영향을 끼쳤는지를 이야기하는 지표이다.
* 회귀분석의 회귀계수 $\Beta$에 해당한다고 할 수 있다.
* 다음과 같은 공식으로 구한다. (회귀분석의 GLS를 차용)

$$
F = ({\Delta}^T {\Psi}^{-1} {\Delta})^{-1} {\Delta}^T {\Psi}^{-1} {X}
$$


In [200]:
# 1번째 인자점수 (GLS로 도출)
n = 1
print(f"{n}번째 인자점수는 다음과 같다.")

f = np.linalg.inv(D.T @ np.linalg.inv(psi) @ D) @ D.T @ np.linalg.inv(psi) @ d1[n-1, :]
print(f)

1번째 인자점수는 다음과 같다.
[-1.23875585 -0.4934509  -0.06052709  0.21970886]


요인분석을 직접 행렬로 풀어보면 다음과 같음.

$$
\begin{pmatrix}
x_{11} & x_{12} & x_{13} & x_{14} \\
x_{21} & x_{22} & x_{23} & x_{24} \\
x_{31} & x_{32} & x_{33} & x_{34} \\
x_{41} & x_{42} & x_{43} & x_{44} \\
\end{pmatrix}
=
\begin{pmatrix}
\delta_{11} & \delta_{12} & \delta_{13} & \delta_{14} \\
\delta_{21} & \delta_{22} & \delta_{23} & \delta_{24} \\
\delta_{31} & \delta_{32} & \delta_{33} & \delta_{34} \\
\delta_{41} & \delta_{42} & \delta_{43} & \delta_{44} \\
\end{pmatrix}
\begin{pmatrix}
f_{1} \\
f_{2} \\
f_{3} \\
f_{4}
\end{pmatrix}
+
\begin{pmatrix}
\epsilon_{1} \\
\epsilon_{2} \\
\epsilon_{3} \\
\epsilon_{4}
\end{pmatrix}
$$

이때, 위에서 파이썬으로 구한 값은 바로 
$\begin{pmatrix}
\delta_{11} & \delta_{12} & \delta_{13} & \delta_{14} \\
\end{pmatrix}
$ 
에 헤당.